# PPP: Global Maps


To do: 1. annual mean predictability horizons, 2. max horizons, 3. range horizons

In [1]:
# Importing packages
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import nc_time_axis
from dask.diagnostics import ProgressBar
import re
from xmovie import Movie
%matplotlib inline
import imageio

In [2]:
gridpath = ('/projects/SOCCOM/data/ESM4_PPE/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/GRID/ocean.static.nc')
grid = xr.open_dataset(gridpath)

In [3]:
rootdir = ('/projects/SOCCOM/data/ESM4_PPE/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/ESM4_piControl_D/gfdl.ncrc4-intel18-prod-openmp/pp/')
ppname = 'ocean_month'
ppname2 = 'ocean_cobalt_omip_2d'
ppname3 = 'ocean_cobalt_omip_sfc'

localdir = '/ts/monthly/5yr/'

In [4]:
# minor helper method - returns the ppname for a particular driver
def pp(driver):
    if driver=='tos' or driver=='sos' or driver=='MLD_003':
        return 'ocean_month'
    elif re.search('intpp*',driver):
        return 'ocean_cobalt_omip_2d'
    elif driver=='chlos':
        return 'ocean_cobalt_omip_sfc'

In [5]:
THRESHOLD = 0.235559205


In [6]:
npp_ppp = xr.open_dataset('/home/saumyam/nppppp.nc')
tos_ppp = xr.open_dataset('/home/saumyam/tosppp.nc')
sos_ppp = xr.open_dataset('/home/saumyam/sosppp.nc')
#sos_ppp = sos_ppp.where(sos_ppp > -100)
mld_ppp = xr.open_dataset('/home/saumyam/mldppp')
#mld_ppp = mld_ppp.where(mld_ppp > -100)
chlos_ppp = xr.open_dataset('/home/saumyam/chlosppp')

In [7]:
def coarse_ppp(driver, driver_ppp):
    annual = driver_ppp.coarsen(time=12).mean()
    coarse_months = xr.where(annual<THRESHOLD,annual.time,120).min(dim='time').where(grid['basin']!=0,np.NaN) # eventually replace this with 120 if it works
    coarse_months[driver].plot()
    return annual

In [ ]:
chlos_mean = coarse_ppp('chlos',chlos_ppp)
plt.title('Surface Chlorophyll, Annual Mean PPP')

In [ ]:
npp_mean = coarse_ppp('intpp',npp_ppp)
plt.title('Net Primary Production, Annual Mean PPP')

In [ ]:
tos_mean = coarse_ppp('tos',tos_ppp)
plt.title('Sea Surface Temperature, Annual Mean PPP')


In [ ]:
sos_mean = coarse_ppp('sos',sos_ppp)
plt.title('Sea Surface Salinity, Annual Mean PPP')


In [ ]:
mld_mean = coarse_ppp('MLD_003',mld_ppp)
plt.title('Mixed Layer Depth, Annual Mean PPP')

In [ ]:
tos_ppp_mean = tos_ppp['tos'].weighted(grid['areacello']).mean(['xh','yh'])
with ProgressBar():
    tos_ppp_mean = tos_ppp_mean.compute()

In [ ]:
tos_ppp_mean.plot()

In [ ]:
tos_ppp['tos'].isel(time=50).plot()

In [ ]:
months = xr.where(tos_ppp<THRESHOLD,tos_ppp.time,120) # eventually replace this with 120 if it works

In [ ]:
max_months = xr.where(tos_ppp>THRESHOLD,tos_ppp.time,np.NaN) # eventually replace this with 120 if it works

In [ ]:
max_npp = xr.where(npp_ppp>THRESHOLD,npp_ppp.time,np.NaN).max(dim='time').where(grid['basin']!=0,np.NaN)

In [ ]:
max_npp['intpp'].plot()

In [ ]:
min_month = months.min(dim='time')

In [ ]:
min_month = min_month.where(grid['basin']!=0,np.NaN)

In [ ]:
max_month = max_months.max(dim='time')

In [ ]:
max_month = max_month.where(grid['basin']!=0,np.NaN)

In [ ]:
max_month['tos'].plot()

In [ ]:
(max_month-min_month)['tos'].plot()

In [ ]:
min_month['tos'].plot()

In [ ]:
sos_months = xr.where(sos_ppp<THRESHOLD,sos_ppp.time,120) # eventually replace this with 120 if it works

sos_min_month = sos_months.min(dim='time')

sos_min_month = sos_min_month.where(grid['basin']!=0,np.NaN)

sos_min_month['sos'].plot()

In [ ]:
npp_months = xr.where(npp_ppp<THRESHOLD,npp_ppp.time,60)
min_npp_month = npp_months.min(dim='time')

In [ ]:
min_npp_month

In [ ]:
new = min_npp_month.where(grid['basin']!=0,np.NaN)

In [ ]:
new['intpp'].plot()

In [ ]:
# can try using positional args: https://xarray.pydata.org/en/stable/generated/xarray.where.html

In [7]:
# returns the variance for a particular ensemble start year
def ens_var(start_year, driver_global, driver):
    # create a list of xarrays of control + ensemble data to be concatenated
    members = [*range(10)]
    ppname = pp(driver)
    
    # handle the control separately
    end = '0' + str(int(start_year) + 9)
    ctrl_slice = driver_global.sel(time=slice(start_year+'-01-16',end+'-12-16'))
    members[0] = ctrl_slice
    
    # loop through all 9 ensemble members
    for member in range(1,10):
        folder = 'ESM4_piControl_D-ensemble-' + start_year + '0101-0' + str(member)
        path = ('/projects/SOCCOM/data/ESM4_PPE/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/'
                +folder+'/gfdl.ncrc4-intel18-prod-openmp/pp/'+ppname+'/ts/monthly/5yr/'+ppname+'.*'+driver+'.nc')
        ens_mem = xr.open_mfdataset(path)
        members[member] = ens_mem
        
    # combine all ensemble members and control into one xarray
    all_members = xr.concat(members,dim='members')
    
    # compute the variance
    variance = (all_members.std(dim='members')) ** 2
    
    return variance
    


In [8]:
STARTS = ['0123','0161','0185','0208','0230','0269','0300','0326','0359','0381']
def avg_ens_var(driver_global, driver):
    # initialize total to be stddev of first ensemble - year '0123'
    
    ensembles = [*range(10)]
    first = ens_var(STARTS[0],driver_global,driver)
    timei = first.time
    ensembles[0] = first
    
    for ens in range(1,10):
        time_adjusted = ens_var(STARTS[ens],driver_global,driver).assign_coords(time=timei)
        ensembles[ens] = time_adjusted
    
    total = xr.concat(ensembles,dim='ensemble')
    avg = total.mean(dim='ensemble')
    
    return avg


## Plotting

In [ ]:
ppp = xr.open_dataset('/home/saumyam/tosppp.nc')

In [ ]:
southern = ppp.where(grid['basin']==1,drop=True)

In [ ]:
atlantic = ppp.where(grid['basin']==2,drop=True)
atlantic_mean = atlantic['tos'].weighted(grid['areacello']).mean(['xh','yh'])
with ProgressBar():
    atlantic_mean = atlantic_mean.compute()

In [ ]:
indian = ppp.where(grid['basin']==5,drop=True)
indian_mean = indian['tos'].weighted(grid['areacello']).mean(['xh','yh'])
with ProgressBar():
    indian_mean = indian_mean.compute()

In [ ]:
pacific = ppp.where(grid['basin']==3,drop=True)
pacific_mean = pacific['tos'].weighted(grid['areacello']).mean(['xh','yh'])
with ProgressBar():
    pacific_mean = pacific_mean.compute()

In [ ]:
arctic = ppp.where(grid['basin']==4,drop=True)
arctic_mean = arctic['tos'].weighted(grid['areacello']).mean(['xh','yh'])
with ProgressBar():
    arctic_mean = arctic_mean.compute()

In [ ]:
southern_mean = (southern['tos'].weighted(grid['areacello']).mean(['xh','yh']))

In [ ]:
with ProgressBar():
    southern_mean = southern_mean.compute()

In [ ]:
ppp_mean = ppp['tos'].weighted(grid['areacello']).mean(['xh','yh'])
with ProgressBar():
    ppp_mean = ppp_mean.compute()

In [ ]:
figure(figsize=(14,8))
southern_mean.plot(label='Southern Ocean')
atlantic_mean.plot(label='Atlantic Ocean')
pacific_mean.plot(label='Pacific Ocean')
arctic_mean.plot(label='Arctic Ocean')
indian_mean.plot(label='Indian Ocean')
ppp_mean.plot(linewidth=4,label='Global Average')
plt.ylabel('PPP of Sea Surface Temperature')
plt.title('Averaged PPP for Sea Surface Temperature')
plt.legend()

In [ ]:
ppp_mean

### Packaged into one function that just takes the driver name as input:

In [9]:
# returns xarray of ppp
def ppp(driver):
    ds = xr.open_mfdataset(rootdir+pp(driver)+localdir+'*'+driver+'.nc')
    ds = ds.chunk({'time':60})
    
    avg_ens = avg_ens_var(ds,driver)
    with ProgressBar():
        avg_ens = avg_ens.compute() 
    timeindex = np.arange(120)
    avg_ens = avg_ens.assign_coords(time=timeindex)
    
    control_monthly = ds[driver].groupby('time.month')
    control_var_monthly = control_monthly.std() ** 2
    control_var_arr = control_var_monthly.to_numpy()
    repeated = np.tile(control_var_arr, (10,1,1))
    #xarray.repeat
    
    ctrl_var = avg_ens.copy(data={driver:repeated})
    ctrl_var = ctrl_var.assign_coords(time=timeindex)
    
    ratio = avg_ens / ctrl_var
    ppp = 1 - ratio
    return ppp

In [10]:
chlos_ppp = ppp('chlos')

[#####                                   ] | 12% Completed |  6.9s

/home/saumyam/.conda/envs/saumyac/lib/python3.9/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 53.2s


/home/saumyam/.conda/envs/saumyac/lib/python3.9/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [ ]:
mld_ppp = ppp('MLD_003')

In [ ]:
chlos_ppp.to_netcdf('/home/saumyam/chlosppp')
mld_ppp.to_netcdf('/home/saumyam/mldppp')

In [ ]:
sosppp = ppp('sos')

In [ ]:
sosppp.to_netcdf('/home/saumyam/sosppp')

In [ ]:
sosppp = xr.open_dataset('/home/saumyam/sosppp.nc')

In [ ]:
# e.g. driver = 'tos', driver_name = 'Sea Surface Temperature'
def plot_basins(ppp, driver, driver_name, adjust=False):
    if adjust:
        ppp = ppp.where(ppp[driver] > -100)

    means = [*range(5)]
    for i in range(1,6):
        basin = ppp.where(grid['basin']==i,drop=True)
        means[i-1] = (basin[driver].weighted(grid['areacello']).mean(['xh','yh']))

    ppp_mean = ppp[driver].weighted(grid['areacello']).mean(['xh','yh'])

    labels = ['Southern Ocean', 'Atlantic Ocean', 'Pacific Ocean', 'Arctic Ocean', 'Indian Ocean']
    figure(figsize=(14,8))
    for i in range(5):
        means[i].plot(label=labels[i])
   
    ppp_mean.plot(linewidth=4,label='Global Average')
    plt.ylabel('PPP of ' + driver_name)
    plt.title('Averaged PPP of ' + driver_name)
    plt.legend()
    
    return ppp_mean

In [ ]:
sos_ppp_mean = plot_basins(sos_ppp, 'sos', 'Sea Surface Salinity',adjust=True)

In [ ]:
tos_ppp_mean = plot_basins(tos_ppp, 'tos', 'Sea Surface Temperature')

In [ ]:
npp_ppp_mean = plot_basins(npp_ppp,'intpp','Integrated Net Primary Production')

In [ ]:
chlos_ppp_mean = plot_basins(chlos_ppp,'chlos','Surface Chlorophyll')

In [ ]:
mld_ppp_mean = plot_basins(mld_ppp,'MLD_003','Mixed Layer Depth',adjust=False)
plt.savefig('PPP MLD OLD')

In [ ]:
mld_ppp_mean = plot_basins(mld_ppp,'MLD_003','Mixed Layer Depth',adjust=True)

In [ ]:
annuals = [tos_mean,sos_mean,mld_mean,npp_mean,chlos_mean]
drivers = ['tos','sos','MLD_003','intpp','chlos']
annuals_global = [*range(5)]

for i in range(len(annuals)):
    annuals_global[i] = annuals[i][drivers[i]].weighted(grid['areacello']).mean(['xh','yh'])
    
COLORS = ['green','orange','red','blue','purple']

In [ ]:

fig,ax = plt.subplots(1,1,figsize=(13,7))
months=[0,12,24,36,48,60,72,84,96,108,120]
ax.set_xticks(months)

for i in range(len(annuals_global)):
    annuals_global[i].plot(color=COLORS[i],linewidth=2)

npp_ppp_mean.plot(label='Net Primary Production')
sos_ppp_mean.plot(label='Sea Surface Salinity')
tos_ppp_mean.plot(label='Sea Surface Temperature')
mld_ppp_mean.plot(label='Mixed Layer Depth')
chlos_ppp_mean.plot(label='Surface Chlorophyll')
plt.axhline(y=THRESHOLD, color='grey', linestyle='dashed',label='Predictability Threshold')
plt.title('Global Average of PPP',fontsize=20)
plt.xlabel('Lead Time (Months)',fontsize=15)
plt.ylabel('PPP',fontsize=15)
plt.legend()
plt.savefig('Globally Averaged PPP-5 drivers')

In [ ]:
chlos_ppp_mean

In [ ]:
npp_ppp.to_netcdf('/home/saumyam/nppppp')

# GIFs

In [ ]:
sosmov = Movie(sos_ppp['sos'],vmin=0,vmax=1)
sosmov.save('sosppp_slow.gif',gif_framerate=4)

In [11]:
sosmov = Movie(sos_ppp['sos'],vmin=0,vmax=1)
sosmov.save('sosppp.mp4',framerate=5)

Movie created at sosppp.mp4


In [18]:
#tos_ppp = xr.open_dataset('/home/saumyam/tosppp.nc')
tosmov = Movie(tos_ppp['tos'],vmin=0,vmax=1)

In [ ]:
tosmov.save('tosppp_slow.gif',gif_framerate=4)

In [19]:
tosmov.save('tosppp.mp4',framerate=5)

Movie created at tosppp.mp4


In [ ]:
nppmov = Movie(npp_ppp['intpp'],vmin=0,vmax=1)
nppmov.save('nppppp_slow.gif',gif_framerate=4)

In [17]:
nppmov = Movie(npp_ppp['intpp'],vmin=0,vmax=1)
nppmov.save('nppppp.mp4',framerate=5,overwrite_existing=True)

Movie created at nppppp.mp4


In [ ]:
mldmov = Movie(mld_ppp['MLD_003'],vmin=0,vmax=1)
mldmov.save('mldppp_slow.gif',gif_framerate=4)

In [20]:
mldmov = Movie(mld_ppp['MLD_003'],vmin=0,vmax=1)
mldmov.save('mldppp.mp4',framerate=5,overwrite_existing=True)

Movie created at mldppp.mp4


In [ ]:
chlosmov = Movie(chlos_ppp['chlos'],vmin=0,vmax=1)
chlosmov.save('chlosppp_slow.gif',gif_framerate=4)

In [21]:
chlosmov = Movie(chlos_ppp['chlos'],vmin=0,vmax=1)
chlosmov.save('chlosppp.mp4',framerate=5,overwrite_existing=True)

Movie created at chlosppp.mp4


In [ ]:
# Combining the gifs
# Code Source: https://stackoverflow.com/questions/51517685/combine-several-gif-horizontally-python
#Create reader object for the gif
gif1 = imageio.get_reader('tosppp.gif')
gif2 = imageio.get_reader('nppppp.gif')
gif3 = imageio.get_reader('sosppp.gif')
gif4 = imageio.get_reader('chlosppp.gif')
#gif5 = imageio.get_reader('mldppp.gif')

#If they don't have the same number of frame take the shorter
number_of_frames = min(gif1.get_length(), gif2.get_length()) 

#Create writer object
new_gif = imageio.get_writer('output.gif')

for frame_number in range(number_of_frames):
    img1 = gif1.get_next_data()
    img2 = gif2.get_next_data()
    img3 = gif3.get_next_data()
    img4 = gif4.get_next_data()
    img5 = gif5.get_next_data()
    #here is the magic
    new_image = np.hstack((img1, img2))
    new_image2 = np.hstack((img3, img4))
    #newest = np.hstack((np.vstack((new_image, new_image2)), img5))
    new_gif.append_data(newest)

gif1.close()
gif2.close() 
gif3.close()
gif4.close()
gif5.close()
new_gif.close()
